# **Attendance Data Consolidation and Transformation with Pandas**

**Note:** This notebook contains a modified version of codes I developed to improve an existing process during my previous employment. The techniques and data formats applied here are similar to those used in the real-world scenario, but the actual data, including names, numbers, and values, have been randomly generated for the purpose of illustration and confidentiality. The modifications aim to showcase the approach and techniques applied in the original context.

### Goal

The goal is to generate two outputs based on the data sources:
1. Consolidated schedule data
2. Employee login logout data

### Data Sources

#### Schedule data

Schedule data is provided separately by each department but are all Excel files with the same format.

![picture](https://drive.google.com/uc?export=view&id=1OsR9ezvDlW5Pw3ORYCtt4g9LNmYaK_TD)

#### Phone data

Employees are level 1 and 2 service desk analysts. Their login/logout data are derived from their interactions with the software phone system.

![picture](https://drive.google.com/uc?export=view&id=15MZvNlQopQJdMIvQ3qmwzsnTW-qINl8u)

#### Employee List

The employee list is an Excel file containing employee IDs, Z IDs and phone IDs.

![picture](https://drive.google.com/uc?export=view&id=1_as6A6QaJtbvSfLDHSRXP7j57ovyaJ-E)
